In [ ]:
# Imports

import numpy as np
from MountainCarEnv import MountainCarEnv
import matplotlib.pyplot as plot

In [ ]:
# Global vars

env = MountainCarEnv(render_mode="rbg_array")

pos_space = np.linspace(env.min_position, env.max_position, 25)
vel_space = np.linspace(-env.max_speed, env.max_speed, 25)
Q = np.zeros((len(pos_space), len(vel_space), env.action_space.n))


In [ ]:
# Functions

def epsilon_greedy_policy(state, Q, epsilon):
    explore = np.random.binomial(1, epsilon)
    if explore:
        action = env.action_space.sample()
    else:
        action = np.argmax(Q[state[0], state[1]])
    return action

def get_state(obs):
    pos, vel = obs
    pos_bin = np.digitize(pos, pos_space)
    vel_bin = np.digitize(vel, vel_space)
    return pos_bin, vel_bin

def run_mountain_car(Q, epsilon, alpha, gamma, test=False):
    done = False
    totalReward = 0
    obs = env.reset()
    state = get_state(obs)
    while not done:
        action = epsilon_greedy_policy(state, Q, epsilon)
        obs, reward, done, _ = env.step(action)
        totalReward += reward
        newState = get_state(obs)
        if not test:
            delta = alpha * (reward + gamma * np.max(Q[newState[0], newState[1]]) - Q[state[0], state[1], action])
            Q[state[0], state[1], action] += delta
        state = newState
    return totalReward



In [ ]:
# Training

numOfEpisodes = 2000000
printInterval = 10000

epsilon=1
minEpsilon = 0.01
reduction = 2*epsilon/numOfEpisodes

alpha=0.1
gamma=0.99

totalRewards = np.zeros(numOfEpisodes)
totalQ = np.zeros(numOfEpisodes)

for i in range(numOfEpisodes):
    totalReward = run_mountain_car(Q, epsilon, alpha, gamma)
    totalRewards[i] = totalReward
    totalQ[i] = np.mean(Q)
    if epsilon > minEpsilon:
        epsilon -= reduction
    if (i+1) % printInterval == 0:
        print(f"Episode {i+1} out of {numOfEpisodes}; Reward {totalReward}; Epsilon {epsilon}")




In [ ]:
# Training plots

continuousInterval = 10000
avgRewards = np.zeros((int)(numOfEpisodes/continuousInterval))
avgQ = np.zeros((int)(numOfEpisodes/continuousInterval))

for i in range(numOfEpisodes):
    if (i+1) % continuousInterval == 0:
        index = (int)(i/continuousInterval)
        avgQ[index] = np.mean(totalQ[i+1-continuousInterval:i+1])
        avgRewards[index] = np.mean(totalRewards[i+1-continuousInterval:i+1])



plot.plot(avgRewards)
plot.xlabel("Time")
plot.ylabel("Reward")
plot.title("Reward vs Time")
plot.show()
plot.close()

plot.plot(totalRewards)
plot.xlabel("Episode")
plot.ylabel("Reward")
plot.title("Reward vs Episode")
plot.show()
plot.close()


plot.plot(avgQ)
plot.xlabel("Time")
plot.ylabel("Q")
plot.title("Q vs Time")
plot.show()
plot.close()

plot.plot(totalQ)
plot.xlabel("Episode")
plot.ylabel("Q")
plot.title("Q vs Episode")
plot.show()
plot.close()




In [ ]:
# Testing

numOfTests = 200000
printTestInterval = 10000
totalTestingRewards = np.zeros(numOfTests)

for i in range(numOfTests):
    totalTestingRewards[i] = run_mountain_car(Q, 0, alpha, gamma, True)
    if (i+1) % printTestInterval == 0:
        print(f"Test {i+1} out of {numOfTests}; Reward {totalTestingRewards[i]}")




In [ ]:
# Testing plots
continuousTestingInterval = 1000
avgTestingRewards = np.zeros((int)(numOfTests/continuousTestingInterval))

for i in range(numOfTests):
    if (i+1) % continuousTestingInterval == 0:
        index = (int)(i/continuousTestingInterval)
        avgTestingRewards[index] = np.mean(totalTestingRewards[i+1-continuousTestingInterval:i+1])


plot.plot(avgTestingRewards)
plot.xlabel("Time")
plot.ylabel("Reward")
# plot.ylim(-200, 0)
plot.title("Reward vs Time")
plot.show()
plot.close()

plot.plot(totalTestingRewards)
plot.xlabel("Episode")
plot.ylabel("Reward")
# plot.ylim(-200, 0)
plot.title("Reward vs Episode")
plot.show()
plot.close()